# Experiment 05-04

In [1]:
import sys
import numpy as np
import hashlib
from binascii import hexlify
import pyopencl as cl
from Library.opencl_information import opencl_information
import logging

## Configure Logging

In [2]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
        logging.FileHandler("ex05_04.log"),
        logging.StreamHandler(sys.stdout)
    ])

logging.info("Started")

2022-07-11 16:16:54 INFO     Started


## Show the available Platforms

In [3]:
info = opencl_information()
info.print_full_info()


OpenCL Platforms and Devices
Platform 0 - Name: Apple
Platform 0 - Vendor: Apple
Platform 0 - Version: OpenCL 1.2 (Apr 19 2022 18:44:44)
Platform 0 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: Apple M1
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1000 Mhz
 Device - Compute Units: 8
 Device - Local Memory: 32 KB
 Device - Constant Memory: 1048576 KB
 Device - Global Memory: 5 GB
 Device - Max Buffer/Image Size: 1024 MB
 Device - Max Work Group Size: 256




## Configure the OpenCL Context

In [4]:
platform_number = 0
device_number = 0

cl_devices = cl.get_platforms()[platform_number].get_devices()
cl_ctx = cl.Context(cl_devices)
cl_queue = cl.CommandQueue(cl_ctx, cl_devices[device_number])

## Compile the Program

In [5]:
def build_program(program_files : list, cl_ctx : cl.Context,
        build_options=[]) -> cl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    cl_ctx : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source = ''

    for cl_file in program_files:
        with open(cl_file, 'r') as cl_file:
            file_source = cl_file.read()
            program_source += '\n' + file_source

    program_source = cl.Program(cl_ctx, program_source)
    program = program_source.build(options=build_options)
            
    return program

In [6]:
cl_program_files = [
    'Library/worker/sha256.cl',
    'Library/worker/zimcoin.cl',
]

cl_program = build_program(cl_program_files, cl_ctx)

# show the kernel names
program_kernel_names = cl_program.get_info(cl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

Kernel Names: hash_main;get_single_hash;get_random_numbers;get_random_string;get_single_hash_nonce;mine_nonce


## Mine Nonce

In [7]:
def display_output(zeros_found : dict, nonce : np.ndarray,
                   nonce_len : np.ndarray) -> None:
    """
    Display the output generated by the OpenCL Kernel.
    """
    for i in range(0, 64):
        if nonce_len[i] > 0:
            if (i not in zeros_found):
                nonce_str = nonce[i * 16:i * 16 + nonce_len[i]].tobytes().decode('UTF-8')
                zeros_found[i] = nonce_str
                hash = hashlib.sha256((plaintext + nonce_str).encode('utf-8'))

                logging.info("%4d: [%2d] %16s %64s" % (i, nonce_len[i], nonce_str, hash.hexdigest() if nonce_len[i] > 0 else ''))

In [8]:
# set up the variables to generate the random numbers
plaintext = 'this is a description of the latest block'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
window_size = np.uint32(1000000)
nonce = np.zeros(shape=16 * 64, dtype=np.uint8)
nonce_len = np.zeros(shape=64, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_window_size = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=window_size)
cl_plaintext_bytes = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_bytes)
cl_plaintext_length = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_length)
cl_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_nonce_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce_len.nbytes)

zeros_found = {}

keep_running = True
while (keep_running):
    seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
    cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)

    # execute the program
    cl_program.mine_nonce(
        cl_queue, (256,), None,
        cl_seed,
        cl_window_size,
        cl_plaintext_bytes,
        cl_plaintext_length,
        cl_nonce,
        cl_nonce_len)

    # get the results
    cl.enqueue_copy(cl_queue, nonce, cl_nonce)
    cl.enqueue_copy(cl_queue, nonce_len, cl_nonce_len)

    # display the results
    display_output(zeros_found, nonce, nonce_len)

    keep_running = True

2022-07-11 16:16:54 INFO        1: [ 1]                S 0486fe740a12e3b54c778665ddcc98a056b0f16570558f9dcf218de51b984d20
2022-07-11 16:16:54 INFO        2: [12]     ePkwj}p6=Ng< 00aa28239599ae0e37662d8226a9af14e538c1793f7457f191d3bfb0e615c1a8
2022-07-11 16:16:54 INFO        3: [ 8]         OsL"|7FO 0006b65b1e0d43b5b0f426f9c69e7f615011d5175c2452a8e20208108f21c820
2022-07-11 16:16:54 INFO        4: [13]    C-ugwEbP2=sxi 0000b85ace7835b821db4271acef2b94250baf4e88b701b0814a2701c010ecb7
2022-07-11 16:17:09 INFO        5: [15]  2=%wjjC.=w4=RC" 000003a88ad16081c0a666563a2fd47973ef84e9d2f18e80239db49f135887d0
